Challenge: ccv1  
# Deep Learning: Was versteckt sich da?  
# CSV mit Bounding Boxen erstellen
In diesem Notebook werden die Megadetector Json Daten nach den Erkenntnissen im Notebook `eda_megadetector_json` bereinigt und CSV-Files mit den zugehörigen Bounding Boxen erzeugt.

In [ ]:
import json
import ast
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import torchvision.transforms as transforms
from PIL import Image

In [ ]:
save_csv_file = False

## Daten bereinigen

In [ ]:
def read_megadetector_json(path_json:str = 'train_features_output.json'):
    mega_json = json.load(open(path_json))

    df_images = pd.DataFrame(mega_json["images"])
    df_images = df_images.reset_index(drop=True)
    return df_images


def transform_bbox(image=None, normalized_bbox=None, image_size:int=None):
    image_width, image_height = image.size
        
    x, y, width, height = normalized_bbox
    remove_bbox_xy = 4
    remove_bbox_wh = remove_bbox_xy * 2

    x = int(np.round(x * image_width, 0)) + remove_bbox_xy
    y = int(np.round(y * image_height, 0)) + remove_bbox_xy
    width = int(np.round(width * image_width, 0)) - remove_bbox_wh
    height = int(np.round(height * image_height, 0)) - remove_bbox_wh
    return y, x, height, width


def get_correct_box(df, train_data):
    bbox_transformed = []
    if train_data: 
        data_path = r"../competition_data/train_features/"
    else:
        data_path = r"../competition_data/test_features/"

    for image_name in df.index:
        if df.loc[image_name]["bbox_true"]:
            path = data_path + df.loc[image_name]["file"]
            image = Image.open(path).convert("RGB")

            normalized_bbox = df.loc[image_name]['bbox_normalized']
            bbox_transformed.append(transform_bbox(image=image, normalized_bbox=normalized_bbox))
        else:
            bbox_transformed.append(pd.NA)

    df['bbox'] = bbox_transformed
    return df


def get_clean_dataframe_from_json(path_json, save_csv=False, file_name='', train_data=True):
    '''
    filter: filters bbox below threshold if true
    '''

    df_images = read_megadetector_json(path_json)
    dict_detection_cat = {'1': 'animal', '2': 'person', '3': 'vehicle'}

    df_images_detections = df_images['detections'].apply(pd.Series, dtype='object')
    #display(df_images_detections.head(2))

    df_images_clean = df_images.merge(df_images_detections, left_index=True, right_index=True).drop(columns='detections')
    #display(df_images_clean.head(2))

    df_images_clean = df_images_clean.melt(id_vars=['file', 'max_detection_conf']).sort_values('file')
    #display(df_images_clean)

    # remove nan values 
    df_images_clean = df_images_clean.dropna(subset='value')
    df_images_clean = df_images_clean.drop(columns='variable')

    # expand category, conf, bbox
    df_cat_conf_bbox = df_images_clean['value'].apply(pd.Series, dtype='object')
    df_images_clean = df_images_clean.merge(df_cat_conf_bbox, left_index=True, right_index=True).drop(columns='value')

    # take only detections for max detection conf
    df_images_clean = df_images_clean[df_images_clean['max_detection_conf'] == df_images_clean['conf']]
    df_images_clean = df_images_clean.reset_index(drop=True)

    # leftjoin clean data
    df_images_clean = pd.merge(df_images['file'], df_images_clean, on='file', how='left')


    # cast datatypes
    df_images_clean['category'] = df_images_clean['category'].astype('category')
    df_images_clean['category'] = df_images_clean['category'].replace(dict_detection_cat)

    # change index to image name
    df_images_clean['image_name'] = [image.replace('.jpg', '') for image in df_images_clean['file']]
    df_images_clean.index = df_images_clean['image_name']
    df_images_clean = df_images_clean.drop(columns='image_name')

    # remove duplicates
    df_images_clean = df_images_clean[df_images_clean.index.duplicated() == False]

    # bbox
    df_images_clean['bbox_normalized'] = df_images_clean['bbox']
    df_images_clean['bbox_true'] = df_images_clean['bbox_normalized'].notnull()

    # transform bbox
    df_images_clean = get_correct_box(df_images_clean, train_data)
            
    if save_csv: df_images_clean.to_csv(f'{file_name}.csv')

    return df_images_clean

def get_csv_file(path_json, merge_csv='../competition_data/train_features_with_split.csv', save_csv=False, file_name='', train_data=True):
    df_images_clean = get_clean_dataframe_from_json(path_json=path_json, train_data=train_data)
    df_merge = pd.read_csv(merge_csv, index_col='id')

    df_combined = df_merge.merge(df_images_clean, left_index=True, right_index=True, how='left')
    df_combined = df_combined.drop(columns=['file', 'max_detection_conf', 'category'])
    if save_csv: df_combined.to_csv(f'{file_name}.csv')

    return df_combined

def crop_image_to_normalized_bbox(image, normalized_bbox):
    y, x, height, width = transform_bbox(image, normalized_bbox)
    image_tensor = transforms.ToTensor()(image)
    return transforms.ToPILImage()((transforms.functional.crop(image_tensor,y,x,height,width)))

def crop_image_to_bbox(image, bbox):
    y, x, height, width = bbox
    image_tensor = transforms.ToTensor()(image)
    return transforms.ToPILImage()((transforms.functional.crop(image_tensor,y,x,height,width)))


## Trainfeatures CSV

In [ ]:
df_trainfeatures_megadet_bbox = get_csv_file(path_json='train_features_output.json', 
                                            save_csv=save_csv_file, 
                                            file_name='../competition_data/trainfeatures_megadet_bbox_split')
df_trainfeatures_megadet_bbox

## Testfeatures CSV

In [ ]:
df_testfeatures_megadet_bbox = get_csv_file(path_json='test_features_output.json',
                                            merge_csv='../competition_data/test_features.csv', 
                                            save_csv=save_csv_file, 
                                            file_name='../competition_data/testfeatures_megadet_bbox',
                                            train_data=False)
df_testfeatures_megadet_bbox

## Test Crop Bilder

Train Daten

In [ ]:
id = 16102

df_train = pd.read_csv('../competition_data/trainfeatures_megadet_bbox_split.csv')

path = r"../competition_data/" + df_train.iloc[id]["filepath"]
image = Image.open(path).convert("RGB")
normalized_bbox = ast.literal_eval(df_train.iloc[id]['bbox_normalized'])
bbox = ast.literal_eval(df_train.iloc[id]['bbox'])

cropped_image = crop_image_to_bbox(image, bbox)
cropped_image_norm = crop_image_to_normalized_bbox(image, normalized_bbox)

display(image)
display(cropped_image)
display(cropped_image_norm)

Test Daten

In [ ]:
id = 512

df_train = pd.read_csv('../competition_data/testfeatures_megadet_bbox.csv')

path = r"../competition_data/" + df_train.iloc[id]["filepath"]
image = Image.open(path).convert("RGB")
normalized_bbox = ast.literal_eval(df_train.iloc[id]['bbox_normalized'])
bbox = ast.literal_eval(df_train.iloc[id]['bbox'])

cropped_image = crop_image_to_bbox(image, bbox)
cropped_image_norm = crop_image_to_normalized_bbox(image, normalized_bbox)

display(image)
display(cropped_image)
display(cropped_image_norm)